https://pyscipopt.readthedocs.io/_/downloads/en/latest/pdf/

In [1]:
# from pyscipopt import Model
# # Create the SCIP model
# model = Model("MIQCP Example")

# # Define variables
# x = model.addVar("x", vtype="BINARY")
# y = model.addVar("y", vtype="INTEGER", lb=0, ub=10)
# z = model.addVar("z", vtype="CONTINUOUS", lb=-5, ub=5)

# # Set objective function
# model.setObjective(2*x + 3*y - z, sense="maximize")

# # Add constraints
# model.addCons(x + y + z <= 15, "c1")
# model.addCons(x + 2*y + z*z <= 20, "c2") # Quadratic constraint
# model.addCons(x - y >= 1, "c3")

# # Solve the model
# model.optimize()

# print(model.getStatus())

# # Print solution
# if model.getStatus() == "optimal":
#     print("Optimal solution found:")
#     print("x =", model.getVal(x))
#     print("y =", model.getVal(y))
#     print("z =", model.getVal(z))
#     print("Objective value =", model.getObjVal())
# else:
    # print("No solution found or optimization failed.")

Using example from https://pyscipopt.readthedocs.io/en/latest/tutorials/model.html to check that it runs / my environment is set up correctly:

In [2]:
# from pyscipopt import Model
# scip = Model()
# x = scip.addVar(vtype='C', lb=0, ub=None, name='x')
# y = scip.addVar(vtype='C', lb=0, ub=None, name='y')
# z = scip.addVar(vtype='C', lb=0, ub=80, name='z')
# cons_1 = scip.addCons(x + y <= 5, name="cons_1")
# cons_1 = scip.addCons(y + z >= 3, name="cons_2")
# cons_1 = scip.addCons(x + y == 5, name="cons_3")
# scip.setObjective(2 * x + 3 * y - 5 * z, sense="minimize")
# scip.optimize()

# solve_time = scip.getSolvingTime()
# num_nodes = scip.getNTotalNodes() # Note that getNNodes() is only the number of nodes for the current run (resets at restart)
# obj_val = scip.getObjVal()
# for scip_var in [x, y, z]:
#     print(f"Variable {scip_var.name} has value {scip.getVal(scip_var)}")

Now adapting it for my problem:

In [3]:
from pyscipopt import Model, quicksum
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
scip = Model()
scip.redirectOutput() #prints to jupyter notebook

User Inputs:

In [5]:
#Dimensions of BOM items:
a = np.array([5,5,10,9,10]) #longest dimension
b = np.array([4,5,1,5,5]) #mid dimension
c = np.array([0.5,0.5,0.5,0.5,0.5]) #smallest dimension

#Dimensions of UNIQUE stock items available for purchase:
l = np.array([15,15]) #longest dimension
w = np.array([10,15]) #mid dimension
h = np.array([0.5,0.5]) #smallest dimension

#Prices of UNIQUE stock items available for purchase:
prices = [30,40]

#Variable dimension info:
n = len(a) #Total number of items on BOM
m = n * len(l) #Upper limit of stock items we'll have to buy

#Adjust l, w, h to be proper dimensions (using n):
new_l = []
for length in l:
    for i in range(n):
        new_l.append(length)
l = new_l

new_w = []
for width in w:
    for i in range(n):
        new_w.append(width)
w = new_w

new_h = []
for height in h:
    for i in range(n):
        new_h.append(height)
h = new_h

#Adjust dimensions of p / replicate each stock board n-1 times:
p = []
for price in prices:
    for i in range(n):
        p.append(price)

p = np.array(p)

In [6]:
# #Dimensions of BOM items:
# a = np.array([6,5]) #longest dimension
# b = np.array([5,4]) #mid dimension
# c = np.array([0.5,0.5]) #smallest dimension

# #Dimensions of UNIQUE stock items available for purchase:
# l = np.array([10]) #longest dimension
# w = np.array([5]) #mid dimension
# h = np.array([0.5]) #smallest dimension

# #Prices of UNIQUE stock items available for purchase:
# prices = [30]

# #Variable dimension info:
# n = len(a) #Total number of items on BOM
# m = n * len(l) #Upper limit of stock items we'll have to buy

# #Adjust l, w, h to be proper dimensions (using n):
# new_l = []
# for length in l:
#     for i in range(n):
#         new_l.append(length)
# l = new_l

# new_w = []
# for width in w:
#     for i in range(n):
#         new_w.append(width)
# w = new_w

# new_h = []
# for height in h:
#     for i in range(n):
#         new_h.append(height)
# h = new_h

# #Adjust dimensions of p / replicate each stock board n-1 times:
# p = []
# for price in prices:
#     for i in range(n):
#         p.append(price)

# p = np.array(p)

In [7]:
# #Dimensions of BOM items:
# a = np.array([10,10]) #longest dimension
# b = np.array([2.5,2.5]) #mid dimension
# c = np.array([0.5,0.5]) #smallest dimension

# #Dimensions of UNIQUE stock items available for purchase:
# l = np.array([10]) #longest dimension
# w = np.array([5]) #mid dimension
# h = np.array([0.5]) #smallest dimension

# #Prices of UNIQUE stock items available for purchase:
# prices = [30]

# #Variable dimension info:
# n = len(a) #Total number of items on BOM
# m = n * len(l) #Upper limit of stock items we'll have to buy

# #Adjust l, w, h to be proper dimensions (using n):
# new_l = []
# for length in l:
#     for i in range(n):
#         new_l.append(length)
# l = new_l

# new_w = []
# for width in w:
#     for i in range(n):
#         new_w.append(width)
# w = new_w

# new_h = []
# for height in h:
#     for i in range(n):
#         new_h.append(height)
# h = new_h

# #Adjust dimensions of p / replicate each stock board n-1 times:
# p = []
# for price in prices:
#     for i in range(n):
#         p.append(price)

# p = np.array(p)

In [8]:
# #Dimensions of BOM items:
# a = np.array([5,5,9,10]) #longest dimension
# b = np.array([4,5,5,5]) #mid dimension
# c = np.array([0.5,0.5,0.5,0.5]) #smallest dimension

# #Dimensions of UNIQUE stock items available for purchase:
# l = np.array([15,15]) #longest dimension
# w = np.array([10,15]) #mid dimension
# h = np.array([0.5,0.5]) #smallest dimension

# #Prices of UNIQUE stock items available for purchase:
# prices = [30,40]

# #Variable dimension info:
# n = len(a) #Total number of items on BOM
# m = n * len(l) #Upper limit of stock items we'll have to buy

# #Adjust l, w, h to be proper dimensions (using n):
# new_l = []
# for length in l:
#     for i in range(n):
#         new_l.append(length)
# l = new_l

# new_w = []
# for width in w:
#     for i in range(n):
#         new_w.append(width)
# w = new_w

# new_h = []
# for height in h:
#     for i in range(n):
#         new_h.append(height)
# h = new_h

# #Adjust dimensions of p / replicate each stock board n-1 times:
# p = []
# for price in prices:
#     for i in range(n):
#         p.append(price)

# p = np.array(p)

#https://pyscipopt.readthedocs.io/en/latest/tutorials/vartypes.html#variable-types

Decision Variables:

In [9]:
u = np.zeros((n, m), dtype=object)
for i in range(n):
    for j in range(m):
        u[i][j] = scip.addVar(vtype='B', name=f"u_{i}_{j}")

x = np.zeros((n,),dtype=object)
for i in range(n):
    x[i] = scip.addVar(vtype='C',lb=0, ub=None, name=f"x_{i}")

y = np.zeros((n,),dtype=object)
for i in range(n):
    y[i] = scip.addVar(vtype='C',lb=0, ub=None, name=f"y_{i}")

r = np.zeros((n,),dtype=object)
for i in range(n):
    r[i] = scip.addVar(vtype='B', name=f"r_{i}")

Intermediate Variables:

In [10]:
q = np.zeros((m,),dtype=object)
for j in range(m):
    q[j] = scip.addVar(vtype='B', name=f"q_{j}")

s = np.zeros((n, n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            s[i][k] = scip.addVar(vtype='B', name=f"s_{i}_{k}")

t = np.zeros((n, n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            t[i][k] = scip.addVar(vtype='B', name=f"t_{i}_{k}")

v = np.zeros((n, n), dtype=object)
for i in range(n):
    for k in range(n):
        v[i][k] = scip.addVar(vtype='B', name=f"v_{i}_{k}")

d = np.zeros((n, n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            d[i][k] = scip.addVar(vtype='B', name=f"d_{i}_{k}")

f = np.zeros((n, n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            f[i][k] = scip.addVar(vtype='B', name=f"f_{i}_{k}")

g = np.zeros((n, n), dtype=object)
for i in range(n):
    for k in range(n):
        g[i][k] = scip.addVar(vtype='B', name=f"g_{i}_{k}")

Constraints:

In [11]:
#1. All BOM items must be cut exactly once / from exactly one stock board:
constr_1 = np.zeros((n,), dtype=object)
for i in range(n):
    constr_1[i] = scip.addCons(quicksum(u[i][j] for j in range(m)) == 1, name=f"constr_1_{i}")

In [12]:
#2. The thickness (smallest dimension) of each BOM item must match that of the stock item from which it's cut
#2a.
constr_2a = np.zeros((n,m), dtype=object)
for i in range(n):
    for j in range(m):
        constr_2a[i][j] = scip.addCons(u[i][j] <= c[i]/h[j], name=f"constr_2a_{i}_{j}")

#2b.
constr_2b = np.zeros((n,m), dtype=object)
for i in range(n):
    for j in range(m):
        constr_2b[i][j] = scip.addCons(u[i][j] <= h[j]/c[i], name=f"constr_2b_{i}_{j}")

In [13]:
#3. If any BOM items are planned to be cut from stock board j, we must buy stock board j:
constr_3 = np.zeros((m,), dtype=object)
for j in range(m):
    constr_3[j] = scip.addCons(q[j] >= quicksum(u[i][j] for i in range(n))/n, name=f"constr_3_{j}")

In [14]:
#4. BOM items cannot exceed the boundaries of the stock board from which they're cut:
constr_4a = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4a[i][k] = scip.addCons(v[i][k] == (s[i][k] + t[i][k]), name=f"constr_4a_{i}_{k}")
        elif i == k:
            constr_4a[i][k] = scip.addCons(v[i][k] == 0, name=f"constr_4a_{i}_{k}")

constr_4c = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4c[i][k] = scip.addCons(s[i][k] <= (y[k] + 1) / (y[i] + (1-r[i])*a[i] + r[i]*b[i]+1), name=f"constr_4c_{i}_{k}")

constr_4d = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4d[i][k] = scip.addCons(s[i][k] >= y[k] - (y[i] + (1-r[i])*a[i] + r[i]*b[i]), name=f"constr_4d_{i}_{k}")

#https://pyscipopt.readthedocs.io/en/stable/tutorials/expressions.html#absolute-abs
constr_4e = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4e[i][k] = scip.addCons(s[i][k] >= 1/100 - abs(y[k] - (y[i] + (1-r[i])*a[i] + r[i]*b[i])), name=f"constr_4e_{i}_{k}")

constr_4g = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4g[i][k] = scip.addCons(t[i][k] <= (y[i] + 1) / (y[k] + (1-r[k])*a[k] + r[k]*b[k]+1), name=f"constr_4g_{i}_{k}")

constr_4h = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4h[i][k] = scip.addCons(t[i][k] >= y[i] - (y[k] + (1-r[k])*a[k] + r[k]*b[k]), name=f"constr_4h_{i}_{k}")

constr_4i = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4i[i][k] = scip.addCons(t[i][k] >= 1/100 - abs(y[i] - (y[k] + (1-r[k])*a[k] + r[k]*b[k])), name=f"constr_4i_{i}_{k}")

constr_4j = np.zeros((m,n,n), dtype=object)
for j in range(m):
    for ii in range(n):
        constr_4j[j][ii] = scip.addCons(w[j] >=  quicksum(u[i][j]*((1-r[i])*b[i] + r[i]*a[i])*(1-v[ii][i]) for i in range(n)), name=f"constr_4j_{j}_{ii}")

constr_4k = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4k[i][k] = scip.addCons(g[i][k] == (d[i][k] + f[i][k]), name=f"constr_4k_{i}_{k}")
        elif i == k:
            constr_4k[i][k] = scip.addCons(g[i][k] == 0, name=f"constr_4k_{i}_{k}")

constr_4m = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4m[i][k] = scip.addCons(d[i][k] <= (x[k] + 1) / (x[i] + (1-r[i])*b[i] + r[i]*a[i]+1), name=f"constr_4m_{i}_{k}")

constr_4n = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4n[i][k] = scip.addCons(d[i][k] >= x[k] - (x[i] + (1-r[i])*b[i] + r[i]*a[i]), name=f"constr_4n_{i}_{k}")

constr_4o = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4o[i][k] = scip.addCons(d[i][k] >= 1/100 - abs(x[k] - (x[i] + (1-r[i])*b[i] + r[i]*a[i])), name=f"constr_4o_{i}_{k}")

constr_4q = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4q[i][k] = scip.addCons(f[i][k] <= (x[i] + 1) / (x[k] + (1-r[k])*b[k] + r[k]*a[k]+1), name=f"constr_4q_{i}_{k}")

constr_4r = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4r[i][k] = scip.addCons(f[i][k] >= x[i] - (x[k] + (1-r[k])*b[k] + r[k]*a[k]), name=f"constr_4r_{i}_{k}")

constr_4s = np.zeros((n,n), dtype=object)
for i in range(n):
    for k in range(n):
        if i != k:
            constr_4s[i][k] = scip.addCons(f[i][k] >= 1/100 - abs(x[i] - (x[k] + (1-r[k])*b[k] + r[k]*a[k])), name=f"constr_4s_{i}_{k}")

constr_4t = np.zeros((m,n,n), dtype=object)
for j in range(m):
    for k in range(n):
        constr_4t[j][k] = scip.addCons(l[j] >=  quicksum(u[i][j]*((1-r[i])*a[i] + r[i]*b[i])*(1-g[k][i]) for i in range(n)), name=f"constr_4t_{j}_{k}")

In [15]:
#5. BOM items cannot overlap each other:
constr_5 = np.zeros((m,n), dtype=object)
for j in range(m):
    for k in range(n):
        constr_5[j][k] = scip.addCons(quicksum(u[i][j] * u[k][j] * (1-v[i][k]) * (1-g[i][k]) for i in range(n) if i != k) == 0, name=f"constr_5_{j}_{k}")

In [16]:
#6. Enforce Graph Coordinates:
constr_6a = np.zeros((n,m), dtype=object)
for i in range(n):
    for j in range(m):
        constr_6a[i][j] = scip.addCons(u[i][j] * (x[i] + (1-r[i])*b[i] + r[i]*a[i]) <= w[j], name=f"constr_6a_{i}_{j}")

constr_6b = np.zeros((n,m), dtype=object)
for i in range(n):
    for j in range(m):
        constr_6b[i][j] = scip.addCons(u[i][j] * (y[i] + (1-r[i])*a[i] + r[i]*b[i]) <= l[j], name=f"constr_6b_{i}_{j}")
        

In [17]:
constraints = scip.getConss()
for constraint in constraints:
    print(scip.printCons(constraint))

  [linear] <constr_1_0>: <u_0_0>[B] +<u_0_1>[B] +<u_0_2>[B] +<u_0_3>[B] +<u_0_4>[B] +<u_0_5>[B] +<u_0_6>[B] +<u_0_7>[B] +<u_0_8>[B] +<u_0_9>[B] == 1None
  [linear] <constr_1_1>: <u_1_0>[B] +<u_1_1>[B] +<u_1_2>[B] +<u_1_3>[B] +<u_1_4>[B] +<u_1_5>[B] +<u_1_6>[B] +<u_1_7>[B] +<u_1_8>[B] +<u_1_9>[B] == 1None
  [linear] <constr_1_2>: <u_2_0>[B] +<u_2_1>[B] +<u_2_2>[B] +<u_2_3>[B] +<u_2_4>[B] +<u_2_5>[B] +<u_2_6>[B] +<u_2_7>[B] +<u_2_8>[B] +<u_2_9>[B] == 1None
  [linear] <constr_1_3>: <u_3_0>[B] +<u_3_1>[B] +<u_3_2>[B] +<u_3_3>[B] +<u_3_4>[B] +<u_3_5>[B] +<u_3_6>[B] +<u_3_7>[B] +<u_3_8>[B] +<u_3_9>[B] == 1None
  [linear] <constr_1_4>: <u_4_0>[B] +<u_4_1>[B] +<u_4_2>[B] +<u_4_3>[B] +<u_4_4>[B] +<u_4_5>[B] +<u_4_6>[B] +<u_4_7>[B] +<u_4_8>[B] +<u_4_9>[B] == 1None
  [linear] <constr_2a_0_0>: <u_0_0>[B] <= 1None
  [linear] <constr_2a_0_1>: <u_0_1>[B] <= 1None
  [linear] <constr_2a_0_2>: <u_0_2>[B] <= 1None
  [linear] <constr_2a_0_3>: <u_0_3>[B] <= 1None
  [linear] <constr_2a_0_4>: <u_0_4>[B] <= 1

Objective Function:

In [18]:
scip.setObjective(quicksum(p[j]*q[j] for j in range(m)), sense="minimize")

In [19]:
scip.optimize()

presolving:
(round 1, fast)       10 del vars, 110 del conss, 0 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 45 clqs
(round 2, fast)       11 del vars, 110 del conss, 0 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 45 clqs
(round 3, exhaustive) 11 del vars, 110 del conss, 0 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 45 upgd conss, 0 impls, 45 clqs
(round 4, exhaustive) 11 del vars, 110 del conss, 1460 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 195 upgd conss, 0 impls, 45 clqs
(round 5, exhaustive) 11 del vars, 110 del conss, 1460 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 295 upgd conss, 0 impls, 3574 clqs
   (1.0s) probing: 1000/1644 (60.8%) - 0 fixings, 0 aggregations, 563648 implications, 0 bound changes
   (1.0s) probing: 1001/1644 (60.9%) - 0 fixings, 0 aggregations, 564426 implications, 0 bound changes
   (1.0s) probing aborted: 1000/1000 successive useless probings
   (1.0s) symmetry compu

In [20]:
solve_time = scip.getSolvingTime()
num_nodes = scip.getNTotalNodes() # Note that getNNodes() is only the number of nodes for the current run (resets at restart)
obj_val = scip.getObjVal()

print("solve_time: ",solve_time)
print("num_nodes: ",num_nodes)
print("obj_val: ",obj_val)

# for scip_var in [x, y, z]:
#     print(f"Variable {scip_var.name} has value {scip.getVal(scip_var)}")

solve_time:  35.0
num_nodes:  1177
obj_val:  40.0


In [21]:
solution = scip.getBestSol()
for var in scip.getVars():
    print(var, ": ",solution[var])

u_0_0 :  0.0
u_0_1 :  -0.0
u_0_2 :  -0.0
u_0_3 :  -0.0
u_0_4 :  -0.0
u_0_5 :  1.0
u_0_6 :  -0.0
u_0_7 :  -0.0
u_0_8 :  -0.0
u_0_9 :  -0.0
u_1_0 :  0.0
u_1_1 :  0.0
u_1_2 :  -0.0
u_1_3 :  -0.0
u_1_4 :  -0.0
u_1_5 :  1.0
u_1_6 :  0.0
u_1_7 :  -0.0
u_1_8 :  -0.0
u_1_9 :  -0.0
u_2_0 :  0.0
u_2_1 :  0.0
u_2_2 :  0.0
u_2_3 :  -0.0
u_2_4 :  -0.0
u_2_5 :  1.0
u_2_6 :  0.0
u_2_7 :  0.0
u_2_8 :  -0.0
u_2_9 :  -0.0
u_3_0 :  0.0
u_3_1 :  0.0
u_3_2 :  0.0
u_3_3 :  0.0
u_3_4 :  -0.0
u_3_5 :  1.0
u_3_6 :  0.0
u_3_7 :  0.0
u_3_8 :  0.0
u_3_9 :  -0.0
u_4_0 :  0.0
u_4_1 :  0.0
u_4_2 :  0.0
u_4_3 :  0.0
u_4_4 :  0.0
u_4_5 :  1.0
u_4_6 :  0.0
u_4_7 :  0.0
u_4_8 :  0.0
u_4_9 :  0.0
r_0 :  1.0
r_1 :  -0.0
r_2 :  0.0
r_3 :  0.0
r_4 :  0.0
q_0 :  0.0
q_1 :  0.0
q_2 :  0.0
q_3 :  0.0
q_4 :  0.0
q_5 :  1.0
q_6 :  0.0
q_7 :  0.0
q_8 :  0.0
q_9 :  0.0
s_0_1 :  0.0
s_0_2 :  1.0
s_0_3 :  1.0
s_0_4 :  1.0
s_1_0 :  -2.3071822230491534e-16
s_1_2 :  1.0
s_1_3 :  1.0
s_1_4 :  1.0
s_2_0 :  0.0
s_2_1 :  0.0
s_2_3 :  0.0
s